In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 66.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Cluster import Butina
import numpy as np
from tqdm import tqdm
import os

# --- 설정 및 파일 매핑 (이 부분은 동일) ---
FP_SCORE_FILE = "selected_ligands3.smi"
PHARMA_SIG_FILE = "pharma_sig_output_parallel.csv"
OUTPUT_DIV_FILE = "diversity_3000_smi_id_only.smi"

ID_COLUMN = 'ID'
SMILES_COLUMN = 'smi'
SIMILARITY_COLUMN = 'Score_FP_main'
DELIMITER = '\t'
TARGET_SIM = 7000
TARGET_DIV = 3000
CLUSTER_THRESHOLD = 0.55 # 임계값 변수는 그대로 사용

# --------------------------------
# 1. 데이터 로드 및 병합 (생략 가능하나 오류 방지를 위해 포함)
# --------------------------------
print("1. 데이터 로드 및 병합 (나머지 분자 풀 확보)")
df_fp = pd.read_csv(FP_SCORE_FILE, sep=DELIMITER, header=None, names=[SMILES_COLUMN, ID_COLUMN, 'Mol_Object_Garbage', SIMILARITY_COLUMN, 'Score_2', 'Score_3', 'Score_4'])
df_fp[SIMILARITY_COLUMN] = pd.to_numeric(df_fp[SIMILARITY_COLUMN], errors='coerce')
df_pharma = pd.read_csv(PHARMA_SIG_FILE)
df_pharma_clean = df_pharma[df_pharma[ID_COLUMN] != ID_COLUMN].copy()
df_final = pd.merge(df_fp[[ID_COLUMN, SMILES_COLUMN, SIMILARITY_COLUMN]], df_pharma_clean[[ID_COLUMN, 'pharma_sig']], on=ID_COLUMN, how='inner')
df_final.dropna(subset=[SIMILARITY_COLUMN], inplace=True)

# --------------------------------
# 2. 상위 7,000개 분자 제외 (Butina 클러스터링 대상 확정)
# --------------------------------
df_sorted = df_final.sort_values(by=SIMILARITY_COLUMN, ascending=False)
df_similarity_7k = df_sorted.head(TARGET_SIM)
df_remaining = df_final[~df_final[ID_COLUMN].isin(df_similarity_7k[ID_COLUMN])].copy()
print(f"   -> Butina 클러스터링 대상 나머지 분자 풀: {len(df_remaining)}개")

# --------------------------------
# 3. Butina 클러스터링 기반 30% 다양성 (3,000개) 선별
# --------------------------------
print("3. Butina 클러스터링 (3,000개 다양성) 시작...")
df_remaining['mol'] = df_remaining[SMILES_COLUMN].apply(Chem.MolFromSmiles)
df_butina_target = df_remaining[df_remaining['mol'].notnull()].reset_index(drop=True)

# Morgan Fingerprint 계산
fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
       for m in tqdm(df_butina_target['mol'], desc="Butina FPs")]

# 거리 계산 (Tanimoto Distance)
dists = []
n = len(fps)
for i in tqdm(range(n), desc="Butina Distances"):
    sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[i+1:])
    dists.extend([1 - s for s in sims])

# ⭐⭐ 수정된 부분: cutoff -> distThresh ⭐⭐
clusters = Butina.ClusterData(dists, nPts=n, distThresh=CLUSTER_THRESHOLD, isDistData=True)
print(f"   -> 생성된 클러스터 수: {len(clusters)}개")

# 클러스터 크기에 비례하여 3,000개 분배 선별
cluster_sizes = [len(c) for c in clusters]
total_size = sum(cluster_sizes)
cluster_select_counts = [
    max(1, int(size / total_size * TARGET_DIV)) for size in cluster_sizes
]

selected_butina_indices = []
for clust, take_n in zip(clusters, cluster_select_counts):
    clust_indices = list(clust)
    np.random.shuffle(clust_indices)
    selected_butina_indices.extend(clust_indices[:take_n])

selected_butina_indices = list(dict.fromkeys(selected_butina_indices))[:TARGET_DIV]
df_diversity_3k = df_butina_target.iloc[selected_butina_indices].copy()
print(f"   -> Butina 다양성 최종 선택: {len(df_diversity_3k)}개")

# --------------------------------
# 4. 30% 다양성 분자만 SMI ID 형태로 저장
# --------------------------------
df_diversity_3k[[SMILES_COLUMN, ID_COLUMN]].to_csv(OUTPUT_DIV_FILE, sep=DELIMITER, index=False, header=False)

print("-" * 30)
print(f"⭐ 최종 3,000개 다양성 후보가 SMI ID 형태로 '{OUTPUT_DIV_FILE}' 파일에 저장 완료! 🎉")

1. 데이터 로드 및 병합 (나머지 분자 풀 확보)
   -> Butina 클러스터링 대상 나머지 분자 풀: 23259개
3. Butina 클러스터링 (3,000개 다양성) 시작...


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATION WARNING: please use MorganGenerator
[23:49:00] DEPRECATI

   -> 생성된 클러스터 수: 1626개
   -> Butina 다양성 최종 선택: 3000개
------------------------------
⭐ 최종 3,000개 다양성 후보가 SMI ID 형태로 'diversity_3000_smi_id_only.smi' 파일에 저장 완료! 🎉


# 새 섹션

In [ ]:
import pandas as pd
import numpy as np

# --- 파일 설정 ---
FILE_7K = "/content/top_7060smi_id_only.smi"
FILE_3K = "diversity_3000_smi_id_only.smi"
DELIMITER = '\t'
COLUMN_NAMES = ['smi', 'ID']
ID_COLUMN = 'ID'

def load_ids(filename):
    """지정된 파일에서 ID를 로드하고 Set으로 반환합니다. 오류 발생 시 None을 반환합니다."""
    try:
        # 파일 로드 및 ID 추출
        df = pd.read_csv(filename, sep=DELIMITER, header=None, names=COLUMN_NAMES)
        ids = set(df[ID_COLUMN].astype(str).str.strip())
        print(f"   -> {filename} ID 수: {len(ids)}개")
        return ids
    except FileNotFoundError:
        print(f"🚨 오류: 파일을 찾을 수 없습니다. 파일명을 확인해 주세요: {filename}")
        return None
    except Exception as e:
        print(f"🚨 파일 로드 중 예상치 못한 오류 발생 ({filename}): {e}")
        return None

# --------------------------------
# 1. 파일 로드 및 ID 추출
# --------------------------------
print("1. 파일 로드 및 ID 추출 시작...")

ids_7k = load_ids(FILE_7K)
ids_3k = load_ids(FILE_3K)

# 파일 로드 성공 여부 확인
if ids_7k is None or ids_3k is None:
    print("\n⚠️ 파일 로드에 실패하여 중복 확인을 진행할 수 없습니다. 파일 경로 및 이름을 확인해주세요.")
else:
    # --------------------------------
    # 2. 중복 ID (교집합) 확인
    # --------------------------------
    print("\n2. 중복 ID (교집합) 확인...")

    # 두 집합의 교집합을 계산 (O(N+M) 속도)
    duplicate_ids = ids_7k.intersection(ids_3k)
    num_duplicates = len(duplicate_ids)

    # --------------------------------
    # 3. 결과 출력 및 저장
    # --------------------------------
    print("-" * 40)
    if num_duplicates == 0:
        print(f"✅ 중복되는 분자 ID는 **총 0개**입니다. (두 파일은 완벽하게 분리되었습니다.)")
    else:
        OUTPUT_DUPLICATES_FILE = "duplicate_ids.txt"

        print(f"⚠️ 중복되는 분자 ID가 **총 {num_duplicates}개** 발견되었습니다.")
        print(f"   -> 중복 ID 목록을 '{OUTPUT_DUPLICATES_FILE}'에 저장합니다.")

        # 중복 ID를 파일로 저장
        with open(OUTPUT_DUPLICATES_FILE, 'w') as f:
            for _id in sorted(list(duplicate_ids)):
                f.write(f"{_id}\n")

    print("-" * 40)
    print("작업 완료.")

1. 파일 로드 및 ID 추출 시작...
   -> /content/top_7060smi_id_only.smi ID 수: 7060개
   -> diversity_3000_smi_id_only.smi ID 수: 3000개

2. 중복 ID (교집합) 확인...
----------------------------------------
⚠️ 중복되는 분자 ID가 **총 55개** 발견되었습니다.
   -> 중복 ID 목록을 'duplicate_ids.txt'에 저장합니다.
----------------------------------------
작업 완료.


In [ ]:
import pandas as pd

# --- 파일 설정 ---
FILE_7055 = "/content/top_7060smi_id_only.smi"  # 하위 5개 제거된 파일
FILE_3K = "diversity_3000_smi_id_only.smi" # 3000개 다양성 파일
OUTPUT_MERGED = "final_merged_10000_candidates.smi"

DELIMITER = '\t'
COLUMN_NAMES = ['smi', 'ID']
ID_COLUMN = 'ID'

# --------------------------------
# 1. 두 파일 로드
# --------------------------------
print("1. 두 파일 로드 시작...")

try:
    df_7055 = pd.read_csv(FILE_7055, sep=DELIMITER, header=None, names=COLUMN_NAMES)
    df_3k = pd.read_csv(FILE_3K, sep=DELIMITER, header=None, names=COLUMN_NAMES)

    print(f"   -> 7055 파일 로드 분자 수: {len(df_7055)}개")
    print(f"   -> 3000 파일 로드 분자 수: {len(df_3k)}개")

except FileNotFoundError as e:
    print(f"🚨 오류: 파일을 찾을 수 없습니다. 파일명을 확인해 주세요: {e.filename}")
    exit()
except Exception as e:
    print(f"🚨 파일 로드 중 예상치 못한 오류 발생: {e}")
    exit()

# --------------------------------
# 2. 두 데이터프레임 병합 및 중복 확인/제거
# --------------------------------
print("\n2. 데이터 병합 및 중복 확인/제거...")

# 두 데이터프레임을 수직으로 합칩니다.
df_combined = pd.concat([df_7055, df_3k], ignore_index=True)
initial_combined_count = len(df_combined)

# ID 컬럼을 기준으로 중복을 확인하고 제거합니다.
# keep='first'로 설정하여 중복되는 경우 먼저 합쳐진 파일(7055)의 분자를 남깁니다.
df_final_merged = df_combined.drop_duplicates(subset=[ID_COLUMN], keep='first')

final_merged_count = len(df_final_merged)
duplicates_removed = initial_combined_count - final_merged_count

print(f"   -> 합계 예상 분자 수: {initial_combined_count}개")
print(f"   -> 제거된 중복 분자 수: {duplicates_removed}개")
print(f"   -> 최종 중복 제거 분자 수: {final_merged_count}개")

# --------------------------------
# 3. 최종 파일 저장 및 줄 수 세기
# --------------------------------
print("\n3. 최종 파일 저장...")

df_final_merged.to_csv(
    OUTPUT_MERGED,
    sep=DELIMITER,
    index=False,
    header=False
)

# 최종 파일의 줄 수(분자 개수)는 final_merged_count와 같습니다.
final_lines_count = final_merged_count

print("-" * 40)
print(f"✅ 중복이 제거된 최종 후보군 파일 '{OUTPUT_MERGED}' 저장이 완료되었습니다.")
print(f"⭐ 최종 파일의 **총 줄 수(분자 개수)**는 **{final_lines_count}개**입니다. ⭐")
print("-" * 40)

1. 두 파일 로드 시작...
   -> 7055 파일 로드 분자 수: 7060개
   -> 3000 파일 로드 분자 수: 3000개

2. 데이터 병합 및 중복 확인/제거...
   -> 합계 예상 분자 수: 10060개
   -> 제거된 중복 분자 수: 55개
   -> 최종 중복 제거 분자 수: 10005개

3. 최종 파일 저장...
----------------------------------------
✅ 중복이 제거된 최종 후보군 파일 'final_merged_10000_candidates.smi' 저장이 완료되었습니다.
⭐ 최종 파일의 **총 줄 수(분자 개수)**는 **10005개**입니다. ⭐
----------------------------------------


In [ ]:
len(final_10000_candidates.smi)

26